# Module 6 Section 1 Lab 1 - Cross Validation
Pengembangan model adalah langkah penting dalam siklus hidup *machine learning*. Kita akan melatih kumpulan data dengan berbagai jenis model yang tersedia. Kita perlu memastikan bahwa model apa pun harus berkinerja baik pada data masa depan. Biasanya, estimasi kesalahan model dilakukan setelah melatih model pada kumpulan data latih, yang dikenal dengan evaluasi residual. Hal itu tidak dapat dipercaya karena hanya berfungsi dengan baik pada data latih. Ada kemungkinan model tersebut *underfitting* atau *overfitting* ketika di uji pada data uji sebenarnya. Jadi, kita tidak bisa hanya melihat metrik evaluasi untuk memilih model dengan kinerja terbaik. Teknik evaluasi yang membantu mengetahui seberapa baik model kita dikenal sebagai Cross-Validation.

**Cross Validation** adalah teknik *resampling* yang membuat model kita yakin tentang efesiensi dan performa pada data yang tidak terlihat. Ini merupakan metode untuk mengevaluasi model dengan melatih model pada subset dari kumpulan data input yang tersedia dan mengevaluasinya pada subset yang berbeda dari kumpulan data.

Referensi: https://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('dataset/diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
data.shape

(768, 9)

In [3]:
# Pisahkan kolom feature dan target
X = data.iloc[:,:-1]
y = data.iloc[:,8:]

## Hold Out Validation - Train and Test Split
Metode Holdout cukup mudah dipahami dan dikerjakan. Sampel data dibagi menjadi dua bagian: data latih dan data uji. Sebelum pembagian berlangsung, sampel data diacak sehingga sampel tercampur dan menghasilkan kumpulan data latih yang akurat. Biasanya, rasio data latih dengan data uji adalah 70:30 atau 80:20. Langkah selanjutnya adalah melatih model dengan data latih dan setelah dilatih, model diuji dengan data uji. Salah satu kelemahan metode ini adalah kumpulan data uji dapat berisi karakteristik penting dari keseluruhan pola atau tren data yang dapat terlewatkan. Dan mungkin saja kumpulan latih tidak mewakili keseluruhan sampel data. Metode ini juga dikenal sebagai pendekatan train/test split.


In [5]:
'''
Split arrays or matrices into random train and test subsets
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
'''

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

train = clf.predict(X_train)
test = clf.predict(X_test)

acc_train = accuracy_score(y_train, train)
acc_test = accuracy_score(y_test, test)

print('Akurasi pelatihan:', acc_train)
print('Akurasi pengujian:', acc_test)


Akurasi pelatihan: 1.0
Akurasi pengujian: 0.6875


## K-Fold Cross Validation
Jenis lain dari cross validation adalah K-Fold. Parameter jenis ini adalah $K$ yang mengacu pada jumlah subset atau lipatan yang diperoleh dari sampel data. Langkah pertama adalah melatih model menggunakan seluruh kumpulan data. Langkah kedua adalah membagi sampel data dalam jumlah $K$ himpunan bagian (misalkan, 10). Dari sini, himpunan bagian ini menjadi kumpulan data pengujian yang kemudian digunakan untuk menguji model satu per satu. Jenis ini dianggap sebagai metode validasi yang tidak bias dan inklusif karena melibatkan pelatihan dan pengujian hampir pada setiap subset.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png'></img>

In [26]:
'''
K-Folds cross-validator
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold
'''

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier()

kfold = KFold(n_splits=10)
kfold_train = cross_val_score(clf, X_train, y_train, cv=kfold)

print(kfold_train)
print('\nRata-rata akurasi pelatihan pada K-Fold:', np.mean(kfold_train))
print('Akurasi pengujian:', acc_test)

[0.70689655 0.67241379 0.67241379 0.87931034 0.65517241 0.77586207
 0.64912281 0.75438596 0.64912281 0.71929825]

Rata-rata akurasi pelatihan pada K-Fold: 0.7133998790078644
Akurasi pengujian: 0.6875


## Repeated K-Fold Cross Validation
Mengulangi K-Fold sebanyak $n$ kali. Dapat digunakan ketika perlu menjalankan K-Fold sebanyak $n$ kali sehingga menghasilkan pemisahan yang berbeda di setiap pengulangan.

In [27]:
'''
Repeated K-Fold cross validator
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html#sklearn.model_selection.RepeatedKFold
'''

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier()

re_kfold = RepeatedKFold(n_splits=10, n_repeats=2)
re_kfold_train = cross_val_score(clf, X_train, y_train, cv=re_kfold)

print(re_kfold_train)
print('\nRata-rata akurasi pelatihan pada Repeated K-Fold:', np.mean(re_kfold_train))
print('Akurasi pengujian:', acc_test)

[0.62068966 0.74137931 0.70689655 0.75862069 0.70689655 0.79310345
 0.68421053 0.54385965 0.59649123 0.63157895 0.67241379 0.56896552
 0.60344828 0.63793103 0.70689655 0.79310345 0.68421053 0.70175439
 0.63157895 0.80701754]

Rata-rata akurasi pelatihan pada Repeated K-Fold: 0.6795523290986085
Akurasi pengujian: 0.6875


## Stratified K-Fold Cross Validation
Metode K-Fold bertingkat adalah metode lain yang melibatkan pembagian kumpulan sampel data dalam himpunan bagian atau lipatan $K$. Namun, untuk memastikan tidak ada pembagian data yang bias pada lipatan $K$, maka dilakukan proses stratifikasi untuk menata ulang data sedemikian rupa sehingga setiap lipatan mewakili seluruh data. Khususnya dalam tugas klasifikasi, subset harus merepresentasikan data dari kedua kelas atau semua kelas. Mungkin stratifikasi mengarah pada penempatan data dari semua kelas di semua himpunan bagian sedemikian rupa sehingga himpunan bagian sepenuhnya mewakili data dan mengarah pada performa model yang lebih tinggi.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png'></img>

In [28]:
'''
Stratified K-Folds cross-validator
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
'''

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier()

stra_kfold = StratifiedKFold(n_splits=10)
stra_kfold_train = cross_val_score(clf, X_train, y_train, cv=stra_kfold)

print(stra_kfold_train)
print('\nRata-rata akurasi pelatihan pada Stratified K-Fold:', np.mean(stra_kfold_train))
print('Akurasi pengujian:', acc_test)

[0.70689655 0.68965517 0.68965517 0.62068966 0.67241379 0.74137931
 0.68421053 0.73684211 0.66666667 0.70175439]

Rata-rata akurasi pelatihan pada Stratified K-Fold: 0.691016333938294
Akurasi pengujian: 0.6875


## Leave-P-Out (LPO \ LPOCV)
Sampel data terdiri dari titik data ($n$). Jumlah total titik data ($n$) digunakan untuk memisahkan sekumpulan titik data yang digunakan untuk pengujian. Titik data ini disebut sebagai ($p$). Kumpulan data latih diperoleh dengan menghitung ($n-p$) dan model dilatih sesuai dengan itu. Setelah pelatihan selesai, $p$ titik data digunakan untuk cross validation. Semua kemungkinan kombinasi $p$ diuji pada model sehingga mendapatkan akurasi yang maksimal.

In [43]:
'''
Leave-P-Out cross-validator
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePOut.html#sklearn.model_selection.LeavePOut
'''

from sklearn.model_selection import LeavePOut
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier()

# tentukan nilai p=int. Semakin besar akan semakin lama komputasinya
lpovc = LeavePOut(p=1)
lpovc.get_n_splits(X_train)


576

In [44]:
lpovc_train = cross_val_score(clf, X_train, y_train, cv=lpovc)

print(lpovc_train)
print('\nRata-rata akurasi pelatihan pada LPOVC:', np.mean(lpovc_train))
print('Akurasi pengujian:', acc_test)

[0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.

## Leave One Out (LOO / LOOCV)
Varian dari metode LPOVC. Pada metode ini, $p$ dijaga menjadi $1 (p=1)$ dan titik data $n-p$ digunakan untuk melatih model. Jadi, ketika pelatihan selesai, titik data $p$ atau titik data tunggal digunakan untuk memvalidasi model. Salah satu kelemahan terbesar dari jenis ini adalah bahwa sebagian besar sampel data digunakan untuk melatih model, namun, hanya satu titik data yang digunakan untuk mengevaluasinya.

In [29]:
'''
Leave-One-Out cross-validator
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut
'''

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier()

loovc = LeaveOneOut()
loovc_train = cross_val_score(clf, X_train, y_train, cv=loovc)

print(loovc_train)
print('\nRata-rata akurasi pelatihan pada LOOVC:', np.mean(loovc_train))
print('Akurasi pengujian:', acc_test)

[1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.

Baca selanjutnya: https://www.di.ens.fr/willow/pdfs/2010_Arlot_Celisse_SS.pdf